In [1]:
# Instalamos las librerías necesarias (si no están instaladas)
!pip install PyMuPDF spacy nltk stanza transformers

In [2]:
!python -m spacy download es_core_news_sm

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ------------------------------- ------- 10.5/12.9 MB 59.4 MB/s eta 0:00:01
     ---------------------------------------- 12.9/12.9 MB 55.8 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [3]:
import fitz  # PyMuPDF para leer PDF
import unicodedata
import spacy
import nltk
import stanza
from transformers import AutoTokenizer
import re
import textblob

c:\Users\ACER\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Descargar modelos necesarios
nltk.download('punkt')
stanza.download('es')
nlp_stanza = stanza.Pipeline('es')
nlp_spacy = spacy.load('es_core_news_sm')
tokenizer_bert = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-08-19 20:07:20 INFO: Downloaded file to C:\Users\ACER\stanza_resources\resources.json
2025-08-19 20:07:20 INFO: Downloading default packages for language: es (Spanish) ...
2025-08-19 20:07:22 INFO: File exists: C:\Users\ACER\stanza_resources\es\default.zip
2025-08-19 20:07:27 INFO: Finished downloading models and saved to C:\Users\ACER\stanza_resources
2025-08-19 20:07:27 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-08-19 20:07:27 INFO: Downloaded file to C:\Users\ACER\stanza_resources\resources.json
2025-08-19 20:07:28 INFO: Loading these models for language: es (Spanish):
| Processor    | Package           |
------------------------------------
| tokenize     | combined          

In [5]:
# 1. Leer el archivo PDF
def leer_pdf(ruta):
    with fitz.open(ruta) as doc:
        texto = "\n".join([page.get_text("text") for page in doc])
    return texto

ruta_pdf = "C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP/Clase_3/ejemplos de ensayos sobre tecnología.pdf"
texto = leer_pdf(ruta_pdf)
print("Texto original:\n", texto[:500])

Texto original:
 ejemplos de ensayos sobre tecnología 
Por: Dr. Marvin L. Smith 
Actualizado: 10 de julio de 2024 
En el panorama digital actual, en rápida evolución, la capacidad de articular pensamientos 
sobre el impacto de la tecnología en nuestro mundo es cada vez más importante. Ya sea que 
sea un estudiante que trabaja en una tarea de clase, un investigador que prepara un artículo o 
un profesional que elabora un informe, comprender cómo escribir de manera efectiva sobre 
tecnología con distintas longitud


In [6]:
# 2. Convertir a minúsculas
texto_minusculas = texto.lower()
print("\nTexto en minúsculas:\n", texto_minusculas[:500])


Texto en minúsculas:
 ejemplos de ensayos sobre tecnología 
por: dr. marvin l. smith 
actualizado: 10 de julio de 2024 
en el panorama digital actual, en rápida evolución, la capacidad de articular pensamientos 
sobre el impacto de la tecnología en nuestro mundo es cada vez más importante. ya sea que 
sea un estudiante que trabaja en una tarea de clase, un investigador que prepara un artículo o 
un profesional que elabora un informe, comprender cómo escribir de manera efectiva sobre 
tecnología con distintas longitud


In [7]:
# 3. Normalización Unicode (NFC)
def normalizar_unicode(texto):
    return unicodedata.normalize('NFC', texto)

texto_normalizado = normalizar_unicode(texto_minusculas)
print("\nTexto normalizado:\n", texto_normalizado[:500])


Texto normalizado:
 ejemplos de ensayos sobre tecnología 
por: dr. marvin l. smith 
actualizado: 10 de julio de 2024 
en el panorama digital actual, en rápida evolución, la capacidad de articular pensamientos 
sobre el impacto de la tecnología en nuestro mundo es cada vez más importante. ya sea que 
sea un estudiante que trabaja en una tarea de clase, un investigador que prepara un artículo o 
un profesional que elabora un informe, comprender cómo escribir de manera efectiva sobre 
tecnología con distintas longitud


In [8]:
# Download the necessary data for Spanish tokenization
nltk.download('punkt')
nltk.download('spanish_grammars')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package spanish_grammars to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package spanish_grammars is already up-to-date!


True

In [13]:
# 4. Tokenización con diferentes librerías

def tokenizar_spacy(texto):
    return [token.text for token in nlp_spacy(texto)]

def tokenizar_nltk(texto):
    return nltk.word_tokenize(texto, language='spanish')

def tokenizar_stanza(texto):
    doc = nlp_stanza(texto)
    return [word.text for sent in doc.sentences for word in sent.words]

def tokenizar_bert(texto):
    return tokenizer_bert.tokenize(texto)

In [14]:
# Aplicamos cada método
tokens_spacy = tokenizar_spacy(texto_normalizado)
tokens_nltk = tokenizar_nltk(texto_normalizado)
tokens_stanza = tokenizar_stanza(texto_normalizado)
tokens_bert = tokenizar_bert(texto_normalizado)

Token indices sequence length is longer than the specified maximum sequence length for this model (4127 > 512). Running this sequence through the model will result in indexing errors


In [15]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [16]:
# Mostramos los primeros 20 tokens de cada método
print("\nTokens con SpaCy:", tokens_spacy[:20])
print("\nTokens con NLTK:", tokens_nltk[:20])
print("\nTokens con Stanza:", tokens_stanza[:20])
print("\nTokens con BERT:", tokens_bert[:20])


Tokens con SpaCy: ['ejemplos', 'de', 'ensayos', 'sobre', 'tecnología', '\n', 'por', ':', 'dr', '.', 'marvin', 'l.', 'smith', '\n', 'actualizado', ':', '10', 'de', 'julio', 'de']

Tokens con NLTK: ['ejemplos', 'de', 'ensayos', 'sobre', 'tecnología', 'por', ':', 'dr.', 'marvin', 'l.', 'smith', 'actualizado', ':', '10', 'de', 'julio', 'de', '2024', 'en', 'el']

Tokens con Stanza: ['ejemplos', 'de', 'ensayos', 'sobre', 'tecnología', 'por', ':', 'dr.', 'marvin', 'l.', 'smith', 'actualizado', ':', '10', 'de', 'julio', 'de', '2024', 'en', 'el']

Tokens con BERT: ['ejemplos', 'de', 'ensayos', 'sobre', 'tecnología', 'por', ':', 'd', '##r', '.', 'mar', '##vin', 'l', '.', 's', '##mith', 'actualizado', ':', '10', 'de']


In [17]:
tokens_bert = tokenizer_bert.tokenize(texto_normalizado)

# Dividimos en fragmentos de 512 tokens
max_length = 512
chunks = [tokens_bert[i:i+max_length] for i in range(0, len(tokens_bert), max_length)]

print(f"Se generaron {len(chunks)} fragmentos de 512 tokens")

Se generaron 9 fragmentos de 512 tokens


In [18]:
def tokenizar_bert(texto):
    return tokenizer_bert.tokenize(texto)

In [20]:
print("\nTokens con BERT:", tokens_bert[:30])


Tokens con BERT: ['ejemplos', 'de', 'ensayos', 'sobre', 'tecnología', 'por', ':', 'd', '##r', '.', 'mar', '##vin', 'l', '.', 's', '##mith', 'actualizado', ':', '10', 'de', 'julio', 'de', '202', '##4', 'en', 'el', 'panorama', 'digital', 'actual', ',']


In [21]:
# Eliminar stop words usando SpaCy
def eliminar_stopwords(tokens, nlp):
    return [token for token in tokens if token not in nlp.Defaults.stop_words]

tokens_sin_stopwords = eliminar_stopwords(tokens_spacy, nlp_spacy)
print("\nTokens sin stop words (primeros 20):", tokens_sin_stopwords[:20])


Tokens sin stop words (primeros 20): ['ejemplos', 'ensayos', 'tecnología', '\n', ':', 'dr', '.', 'marvin', 'l.', 'smith', '\n', 'actualizado', ':', '10', 'julio', '2024', '\n', 'panorama', 'digital', 'actual']


In [22]:
import re

# Buscar números en el texto
numeros = re.findall(r'\d+', texto_minusculas)
print("Números encontrados en el texto:", numeros)

Números encontrados en el texto: ['10', '2024', '300', '500', '1000', '300', '500', '1000', '300', '500', '1000', '300', '500', '3', '1000', '3', '42', '2024', '100', '2023', '1736900', '2022', '3', '500', '80', '12', '2024']


In [23]:
for idx, numero in enumerate(numeros, start=1):
    print(f"Número {idx}: {numero}")

Número 1: 10
Número 2: 2024
Número 3: 300
Número 4: 500
Número 5: 1000
Número 6: 300
Número 7: 500
Número 8: 1000
Número 9: 300
Número 10: 500
Número 11: 1000
Número 12: 300
Número 13: 500
Número 14: 3
Número 15: 1000
Número 16: 3
Número 17: 42
Número 18: 2024
Número 19: 100
Número 20: 2023
Número 21: 1736900
Número 22: 2022
Número 23: 3
Número 24: 500
Número 25: 80
Número 26: 12
Número 27: 2024


In [24]:
# Importamos la librería 're', que es el módulo de expresiones regulares de Python.
# No necesita instalación porque viene incluida con el lenguaje.
import re

# Definimos la función que realizará la tokenización.
# Recibe una cadena de texto como entrada.
def tokenizar_con_regex_y_numeros(texto_minusculas):
    """
    Esta función tokeniza un texto utilizando una expresión regular
    diseñada para separar palabras, números y signos de puntuación.
    """
    
    # Definimos el patrón de la expresión regular. Se lee como una serie de "O" lógicos (|).
    # r'' indica que es una "raw string" para que los caracteres como '\' no se interpreten como secuencias de escape.
    # 1. \d+      : Busca una o más ocurrencias consecutivas de dígitos (números). Esto captura '500', '2025', '10'.
    # 2. |         : Actúa como un "O".
    # 3. [A-Za-zÁ-ú]+ : Busca una o más ocurrencias de letras, tanto mayúsculas como minúsculas.
    #                  Incluimos 'Á-ú' para capturar caracteres del español como 'á', 'é', 'í', 'ó', 'ú', 'ñ'.
    # 4. |         : Otro "O".
    # 5. [^\w\s]  : Busca cualquier caracter individual que NO sea (\^) un caracter de palabra (\w, que incluye letras, números y _)
    #                  Y que NO sea (\^) un espacio en blanco (\s). Esto nos permite capturar signos de puntuación como '.', ':', '$'.
    patron = r'\d+|[A-Za-zÁ-ú]+|[^\w\s]'
    
    # Utilizamos la función re.findall() para encontrar todas las subcadenas en el texto
    # que coincidan con nuestro patrón. Devuelve una lista con todos los hallazgos.
    tokens = re.findall(patron, texto_minusculas)
    
    # La función devuelve la lista de tokens encontrados.
    return tokens

In [25]:
# --- Proceso de ejecución ---

# 1. Definimos un texto de ejemplo que contiene palabras, números y puntuación.
# texto_ejemplo = "En 2025, el producto A-42 costará $500.50 y la entrega será a las 10:30."

# 2. Llamamos a nuestra función personalizada para tokenizar el texto.
tokens_resultado = tokenizar_con_regex_y_numeros(texto_minusculas)

# 3. Imprimimos el resultado para verificar cómo se separaron los elementos.
print(f"Texto original:\n{texto_minusculas}\n")
print(f"Tokens generados ({len(tokens_resultado)} tokens):\n{tokens_resultado}")

Texto original:
ejemplos de ensayos sobre tecnología 
por: dr. marvin l. smith 
actualizado: 10 de julio de 2024 
en el panorama digital actual, en rápida evolución, la capacidad de articular pensamientos 
sobre el impacto de la tecnología en nuestro mundo es cada vez más importante. ya sea que 
sea un estudiante que trabaja en una tarea de clase, un investigador que prepara un artículo o 
un profesional que elabora un informe, comprender cómo escribir de manera efectiva sobre 
tecnología con distintas longitudes es una habilidad valiosa. 
este artículo presenta tres ensayos de muestra sobre el tema de la tecnología, cada uno 
adaptado a un número específico de palabras: 300, 500 y 1000 palabras. estos ensayos 
sirven como modelos para demostrar cómo un mismo tema puede abordarse con diferentes 
niveles de profundidad y detalle, dependiendo de la extensión requerida. 
el ensayo de 300 palabras ofrece una descripción general concisa y aborda puntos clave 
sobre la influencia de la tecno

In [26]:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install pdfplumber --no-cache-dir

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ------------------------------ --------- 9.7/12.9 MB 52.7 MB/s eta 0:00:01
     ---------------------------------------- 12.9/12.9 MB 44.9 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [27]:
import pdfplumber  # Para extraer texto de PDFs
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode

In [28]:
nlp = spacy.load("es_core_news_sm")  #cargar el modelo en español

In [29]:
# Importa la librería fitz, que es el nombre del módulo PyMuPDF
import fitz

# Usa la misma ruta de tu archivo
pdf_path = "C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP/Clase_3/ejemplos de ensayos sobre tecnología.pdf"
texto_pdf = ""

try:
    # Abre el documento PDF con fitz
    with fitz.open(pdf_path) as doc:
        # Itera sobre cada página del documento
        for pagina in doc:
            # Extrae el texto de la página y lo añade a la variable
            texto_pdf += pagina.get_text()
    
    # Imprime el texto extraído (o una parte para verificar)
    print("¡Texto extraído con éxito!")
    print(texto_pdf[:500]) # Imprime los primeros 500 caracteres

except Exception as e:
    # Si ocurre un error, imprímelo para saber qué pasó
    print(f"No se pudo leer el PDF. El archivo podría estar dañado o no ser un PDF válido.")
    print(f"Error original: {e}")

¡Texto extraído con éxito!
ejemplos de ensayos sobre tecnología 
Por: Dr. Marvin L. Smith 
Actualizado: 10 de julio de 2024 
En el panorama digital actual, en rápida evolución, la capacidad de articular pensamientos 
sobre el impacto de la tecnología en nuestro mundo es cada vez más importante. Ya sea que 
sea un estudiante que trabaja en una tarea de clase, un investigador que prepara un artículo o 
un profesional que elabora un informe, comprender cómo escribir de manera efectiva sobre 
tecnología con distintas longitud


In [31]:
pdf_path = "C:/Users/ACER/OneDrive/Documentos/Analitica de datos/1. Maestria/7. NLP/Clase_3/ejemplos de ensayos sobre tecnología.pdf"
# Extraer texto del PDF
with pdfplumber.open(pdf_path) as pdf:
    texto_pdf = ""
    for pagina in pdf.pages:
        texto_pdf += pagina.extract_text()

In [32]:
header_pattern = r"TECNOLOGIA Y SUS EJEMPLOS 2024"

In [33]:
def extract_text_without_header(pdf_path):
    extracted_text = []  # Lista donde almacenaremos el texto de cada página

    with pdfplumber.open(pdf_path) as pdf:  # Abrir el archivo PDF
        for page in pdf.pages:  # Iterar por cada página del PDF
            text = page.extract_text()  # Extraer texto de la página
            if text:  # Verificar si hay texto en la página
                text = re.sub(header_pattern, "", text, flags=re.DOTALL)  # Eliminar encabezado con regex
                extracted_text.append(text)  # Guardar el texto limpio en la lista

    return " ".join(extracted_text)  # Unir todas las páginas en un solo texto

In [34]:
def preprocess_text(text):
    text = unicodedata.normalize("NFKD", text)  # Normalizar caracteres Unicode (ej. á → a)

    doc = nlp(text.lower())  # Convertir todo el texto a minúsculas y procesarlo con spaCy

    tokens = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return tokens

In [35]:
raw_text = extract_text_without_header(pdf_path)  # Extraer texto limpio del PDF
tokens = preprocess_text(raw_text)  # Preprocesar el texto con spaCy

print(tokens[:50])  # Mostrar los primeros 50 tokens como prueba

['ejemplos', 'ensayos', 'dr', 'marvin', 'smith', 'actualizado', 'julio', 'panorama', 'digital', 'actual', 'capacidad', 'articular', 'pensamientos', 'impacto', 'mundo', 'importante', 'estudiante', 'trabaja', 'tarea', 'clase', 'investigador', 'prepara', 'profesional', 'elabora', 'informe', 'comprender', 'escribir', 'efectiva', 'distintas', 'longitudes', 'habilidad', 'valiosa', 'presenta', 'ensayos', 'muestra', 'tema', 'adaptado', 'palabras', 'palabras', 'ensayos', 'sirven', 'modelos', 'demostrar', 'tema', 'abordarse', 'niveles', 'profundidad', 'detalle', 'dependiendo', 'requerida']


In [37]:
with open("TALLER_3.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(tokens))  # Guardar los tokens en un archivo de texto
print("Tokens guardados en 'TALLER_3.txt'")

Tokens guardados en 'TALLER_3.txt'
